# US Attraction EDA

In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras as extras
import math

In [2]:
df = pd.read_csv("../data/cleaned_data_USA.csv", index_col=0)

In [3]:
df.head()

,name,main_category,rating,reviews,categories,address,city,country,state,zipcode,broader_category,Weighted_Score,Weighted_Average,All_Cities
0,Forsyth Park,Park,4.8,16538.0,"Park, Tourist attraction","Forsyth Park, Savannah, GA 31401",Savannah,USA,GA,NaN,Nature,79382.4,4.67,"Atlanta, Augusta, Chattanooga, Savannah"
1,The Cathedral Basilica of St. John the Baptist,Catholic cathedral,4.8,5911.0,"Catholic cathedral, Catholic church, Tourist a...",The Cathedral Basilica of St. John the Baptist...,Savannah,USA,GA,NaN,Religious,28372.8,4.80,"Atlanta, Augusta, Chattanooga, Savannah"
2,Fort Pulaski National Monument,Monument,4.8,5221.0,"Monument, Historical place, Historical landmar...","Fort Pulaski National Monument, 101 Fort Pulas...",Savannah,USA,GA,NaN,Cultural,25060.8,4.53,"Atlanta, Augusta, Chattanooga, Savannah"
3,Fountain at Forsyth Park,Historical landmark,4.8,4234.0,"Historical landmark, Tourist attraction","Fountain at Forsyth Park, 1 W Gaston St, Savan...",Savannah,USA,GA,NaN,Cultural,20323.2,4.53,"Atlanta, Augusta, Chattanooga, Savannah"
4,Wormsloe State Historic Site,Historical place museum,4.5,3615.0,"Historical place museum, Museum, Park, State park","Wormsloe State Historic Site, 7601 Skidaway Rd...",Savannah,USA,GA,NaN,Cultural,16267.5,4.53,"Atlanta, Augusta, Chattanooga, Savannah"


In [4]:
conn = psycopg2.connect(
    dbname="postgresdb",
    user="postgres",
    password="postgres_password",
    host="host.docker.internal",  # e.g., "localhost"
    port="5433"        # default PostgreSQL port
)

cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())


('PostgreSQL 16.10 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit',)


In [22]:
df.head()

,name,main_category,rating,reviews,categories,address,city,country,state,zipcode,broader_category,Weighted_Score,Weighted_Average,All_Cities
0,Forsyth Park,Park,4.8,16538.0,"Park, Tourist attraction","Forsyth Park, Savannah, GA 31401",Savannah,USA,GA,NaN,Nature,79382.4,4.67,"Atlanta, Augusta, Chattanooga, Savannah"
1,The Cathedral Basilica of St. John the Baptist,Catholic cathedral,4.8,5911.0,"Catholic cathedral, Catholic church, Tourist a...",The Cathedral Basilica of St. John the Baptist...,Savannah,USA,GA,NaN,Religious,28372.8,4.80,"Atlanta, Augusta, Chattanooga, Savannah"
2,Fort Pulaski National Monument,Monument,4.8,5221.0,"Monument, Historical place, Historical landmar...","Fort Pulaski National Monument, 101 Fort Pulas...",Savannah,USA,GA,NaN,Cultural,25060.8,4.53,"Atlanta, Augusta, Chattanooga, Savannah"
3,Fountain at Forsyth Park,Historical landmark,4.8,4234.0,"Historical landmark, Tourist attraction","Fountain at Forsyth Park, 1 W Gaston St, Savan...",Savannah,USA,GA,NaN,Cultural,20323.2,4.53,"Atlanta, Augusta, Chattanooga, Savannah"
4,Wormsloe State Historic Site,Historical place museum,4.5,3615.0,"Historical place museum, Museum, Park, State park","Wormsloe State Historic Site, 7601 Skidaway Rd...",Savannah,USA,GA,NaN,Cultural,16267.5,4.53,"Atlanta, Augusta, Chattanooga, Savannah"


In [21]:
df.dtypes

name                 object
main_category        object
rating              float64
reviews             float64
categories           object
address              object
city                 object
country              object
state                object
zipcode             float64
broader_category     object
Weighted_Score      float64
Weighted_Average    float64
All_Cities           object
dtype: object

# Postgres Upload

In [5]:
# Create table SQL – example with basic types to match DataFrame columns
create_table_query = '''
CREATE TABLE IF NOT EXISTS us_attractions (
    name VARCHAR(250),
    main_category VARCHAR(250),
    rating REAL,
    reviews REAL,
    categories VARCHAR(250),
    address VARCHAR(250),
    city VARCHAR(250),
    country VARCHAR(250),
    state VARCHAR(250),
    zipcode INTEGER,
    broader_category VARCHAR(250),
    weighted_score REAL,
    weighted_average REAL,
    all_cities VARCHAR(250)
);
'''
cursor.execute(create_table_query)
conn.commit()

In [15]:
len(df.columns)

14

In [6]:
def clean_tuple_for_insert(tup):
    return tuple(None if (isinstance(x, float) and math.isnan(x)) else x for x in tup)


def load_values(conn, df, table):
    tuples = [clean_tuple_for_insert(tuple(x)) for x in df.to_numpy()]
    col_names = [s.lower() for s in df.columns]
    cols = ','.join(col_names)
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [7]:
load_values(conn, df, 'us_attractions')

In [8]:
cursor.close()
conn.close()